In [ ]:
# bibliotecas 
import pandas as pd
import numpy as np
from math import sqrt

#### 0 - Explorando os dados

In [ ]:
filmes = pd.read_csv(r'C:\Users\Marianna\Documents\Curso\Sistema de Recomendacao\dataset\movies.csv')
filmes.head()

In [ ]:
filmes = filmes[['movieId', 'title']]
filmes.columns = ['id_filme', 'titulo']
filmes

In [ ]:
# alguns filmes com ids diferente
filmes.titulo.value_counts() 

In [ ]:
# numero de filmes na base
filmes.titulo.unique().shape[0]

In [ ]:
lista_filmes = ['Toy Story (1995)', 'Lord of the Rings: The Fellowship of the Ring, The (2001)', 'Mortal Kombat (1995)','Pocahontas (1995)','Star Wars: Episode VI - Return of the Jedi (1983)','Brazil (1985)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']

In [ ]:
filmes = filmes.loc[filmes.titulo.isin(lista_filmes)]
filmes

In [ ]:
avaliacoes = pd.read_csv(r'C:\Users\Marianna\Documents\Curso\Sistema de Recomendacao\dataset\ratings.csv')
avaliacoes.head()

In [ ]:
avaliacoes = avaliacoes[['userId', 'movieId', 'rating']]
avaliacoes.columns = ['id_usuario', 'id_filme', 'nota_filme']
avaliacoes.head()

In [ ]:
avaliacoes = avaliacoes.merge(filmes, on='id_filme')
avaliacoes.head()

In [ ]:
# tamanho da tabela
avaliacoes.shape

In [ ]:
# numero de usuario
avaliacoes.id_usuario.unique().shape[0]

In [ ]:
# numero de filmes
avaliacoes.id_filme.unique().shape[0]

In [ ]:
# numero de classificacao por usuario
avaliacoes.id_usuario.value_counts()

In [ ]:
# nota minima e maxima
avaliacoes.nota_filme.min(), avaliacoes.nota_filme.max()

In [ ]:
# reducao da quantidade de usuario
avaliacoes = avaliacoes.loc[avaliacoes.id_usuario.isin([1,7,371, 608,610])]
avaliacoes

In [ ]:
# guardando a informação de lista de usuarios
lista_usuario = list(avaliacoes.id_usuario.unique())
lista_usuario

#### 1 - Matriz de Score (usuario x item)

In [ ]:
# matriz de avaliacoes usuario x itens
scores = avaliacoes.pivot_table(index = ['id_usuario'], values = ['nota_filme'], columns = ['titulo'])
scores.columns = scores.columns.droplevel(0)
scores = scores.rename_axis(None, axis=1)
scores.rename(columns={'':'id_usuario'}, inplace=True)
scores

In [ ]:
scores = scores.fillna(0)
scores

In [ ]:
# guardando a informação de lista de itens
lista_itens = list(scores.columns)
lista_itens

In [ ]:
matriz_scores = scores.to_numpy()
matriz_scores

#### 2 - Matriz de similaridade (usuario x usuario)

In [ ]:
def distancia(a, b):
    zip_b = zip(*b)
    resultado = []
    quadrado_diferenca = 0

    for row_a in a:
        for col_b in zip_b:
            soma = 0
            for nota_a, nota_b in zip(row_a, col_b):
                if (nota_a !=0) & (nota_b !=0):                   
                    quadrado_diferenca = pow(nota_b-nota_a, 2)
                    soma += quadrado_diferenca
            if soma > 0:
                d = 1/(1 + sqrt(soma))
            if soma == 0:
                d = 1
            resultado.append(d)
    return resultado                                                       

In [ ]:
#teste com um usuario
score_usuario_1 = scores.loc[[scores.index[0]]].to_numpy()

sim_usu1_demais = distancia(score_usuario_1, matriz_scores.T)     
sim_usu1_demais

In [ ]:
def similaridade(scores):
    i = 0
    for usuario in list(scores.index):
        scores_usuario = scores.loc[[usuario]].to_numpy()
        
        matriz_scores = scores.to_numpy()
        
        resultado = distancia(scores_usuario, matriz_scores.T)     
        if i == 0:
            base_similaridade = pd.DataFrame(resultado, index = list(scores.index), columns = [usuario])
        else:
            temp = pd.DataFrame(resultado, index = list(scores.index), columns = [usuario])
            base_similaridade = pd.concat([base_similaridade, temp], axis=1)
        i = i + 1
    return base_similaridade

In [ ]:
base_similaridade = similaridade(scores)
base_similaridade

In [ ]:
matriz_similaridade = base_similaridade.to_numpy()
# resolvendo o problema da diagonal principal
np.fill_diagonal(matriz_similaridade, 0)
matriz_similaridade

#### 3 - Recomendação

In [ ]:
matriz_scores

In [ ]:
soma_similares = np.dot(matriz_similaridade, matriz_scores)
soma_similares

In [ ]:
matriz_scores_aux = np.copy(matriz_scores)    
matriz_scores_aux[matriz_scores > 0] = 1
matriz_scores_aux

In [ ]:
soma_pesos = np.dot(matriz_similaridade, matriz_scores_aux)
soma_pesos

In [ ]:
recomendacao = np.around(np.divide(soma_similares, soma_pesos), decimals=2)
recomendacao

In [ ]:
# selecionar somente as notas q foram preditas
recomendacao[matriz_scores_aux == 1] = 0
recomendacao

In [ ]:
recomendacao = pd.DataFrame(recomendacao, index = lista_usuario, columns = lista_itens)
recomendacao

In [ ]:
# trazendo as notas ja classificadas
recomendacao[matriz_scores_aux == 1] = matriz_scores
recomendacao

#### Metodo alternativo do Passo 2 : Matriz de similaridade (para gerar similaridade entre todos os usuarios da base)

In [ ]:
#metodo alternativo para rodar todo o conjunto de dados
from sklearn.metrics.pairwise import pairwise_distances 
matriz_similaridade = pairwise_distances(matriz_scores, metric='cosine')
matriz_similaridade